In [46]:
from bs4 import BeautifulSoup
import requests
import re
#import webbrowser

def get_soup(url):

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:57.0) Gecko/20100101 Firefox/57.0'}

    r = s.get(url, headers=headers)

    #with open('temp.html', 'wb') as f:
    #    f.write(r.content)
    #    webbrowser.open('temp.html')

    if r.status_code != 200:
        print('status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')

def parse(url, response):

    if not response:
        print('no response:', url)
        return

    # get number of reviews
    num_reviews = response.find('span', class_='reviews_header_count').text
    num_reviews = num_reviews[1:-1] # remove `( )`
    num_reviews = num_reviews.replace(',', '') # remove `,`
    num_reviews = int(num_reviews)
    print('num_reviews:', num_reviews, type(num_reviews))

    # create template for urls to pages with reviews
    url = url.replace('.html', '-or{}.html')
    print('template:', url)

    # load pages with reviews
    for offset in range(0, num_reviews, 5):
        print('url:', url.format(offset))
        url_ = url.format(offset)
        parse_reviews(url_, get_soup(url_))
        return # for test only - to stop after first page

def parse_reviews(url, response):
    print('review:', url)

    if not response:
        print('no response:', url)
        return

    # get every review
    for idx, review in enumerate(response.find_all('div', class_='review-container')):
        item = {
            #'hotel_name': response.find('h1', class_='heading_title').text,
            'review_title': review.find('span', class_='noQuotes').text,
            'review_body': review.find('p', class_='partial_entry').text,
            #'review_date': review.find('span', class_='relativeDate')['title'],#.text,#[idx],
            'num_reviews_reviewer': review.find('span', class_='badgetext').text,
            #'reviewer_name': review.find('span', class_='scrname').text,
            #'bubble_rating': review.select_one('div.reviewItemInline span.ui_bubble_rating')['class'][1][7:],
        }

        results.append(item) # <--- add to global list

        #~ yield item
        for key,val in item.items():
            print(key, ':', val)
        print('----')
        # return for test only - to stop after first review


# --- main ---

s = requests.Session()

start_urls = [
    'https://www.tripadvisor.co.uk/Restaurant_Review-g503775-d2519242-Reviews-The_Gurkha_Restaurant-Chelmsford_Essex_England.html'
    #'https://www.tripadvisor.com/Hotel_Review-g562819-d289642-Reviews-Hotel_Caserio-Playa_del_Ingles_Maspalomas_Gran_Canaria_Canary_Islands.html',
    #https://www.tripadvisor.com/Hotel_Review-g60795-d102542-Reviews-Courtyard_Philadelphia_Airport-Philadelphia_Pennsylvania.html',
    #'https://www.tripadvisor.com/Hotel_Review-g60795-d122332-Reviews-The_Ritz_Carlton_Philadelphia-Philadelphia_Pennsylvania.html',
]

results = [] # <--- global list for items

for url in start_urls:
    parse_reviews(url, get_soup(url))

import pandas as pd

df = pd.DataFrame(results) # <--- convert list to DataFrame
df.to_csv('output.csv')    # <--- save in file

review: https://www.tripadvisor.co.uk/Restaurant_Review-g503775-d2519242-Reviews-The_Gurkha_Restaurant-Chelmsford_Essex_England.html
review_title : Lost for words 🙃😕☹️
review_body : Where do I start, used trip advisor to find an India to eat in as not being local we hadn’t a clue, unfortunately trip advisor recommended the Gurkha! 🤔 

1. If u want to be warm get yourself a £800 Canada goose coat or freeze...More
num_reviews_reviewer : 7
----
review_title : Amazing service and even better food
review_body : Headed out for dinner with the family and had such an amazing experience. The food was perfect with lots of flavour and the staff couldn't have been friendlier or more attentive. Can't wait for our next visit!
num_reviews_reviewer : 1
----
review_title : Excellent meal
review_body : During my Christmas holiday in the UK, I visited this restaurant with my son.

We asked the waiter to explain what some of the dishes were and he very gladly helped. I noticed that there were venison dish

In [42]:
results[0].items()

dict_items([('review_title', 'Amazing service and even better food'), ('review_body', "Headed out for dinner with the family and had such an amazing experience. The food was perfect with lots of flavour and the staff couldn't have been friendlier or more attentive. Can't wait for our next visit!"), ('num_reviews_reviewer', '1')])